In [1]:
import numpy as np
import arcode
import codec
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image

In [2]:
def show_data(x_in, pre_in):
    mean = np.mean(x_in)
    std = np.std(x_in)
    Max = np.max(x_in)
    Min = np.min(x_in)
    dtype = x_in.dtype
    
    print(pre_in + ' shape: ' + str(x_in.shape) + ' , mean = ' + str(mean) 
          + ' , std = ' + str(std) + ' , Max = ' + str(Max) + ' , Min = ' + str(Min) 
          + ' , type=' + str(dtype))

In [3]:
def oc_fname(i_in):
    rname = '/f00' + str(i_in) + '.png'
    if i_in >9: rname = '/f0' + str(i_in) + '.png'
    if i_in >99: rname = '/f' + str(i_in) + '.png'
        
    return rname

In [4]:
def read_COMPINT(prefC, cname):
    o4=np.zeros([1, nx, ny, ncomp])
    for k in range(ncomp):
        inname = prefC + str(k) + cname
        #print("  read_COMPINT: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
                
    return o4

In [5]:
def ar_code(x_in, oname):
    img = x_in.flatten()
    olen = len(img)
    img = img.reshape(olen,1,1)
    
    bits = codec.bitstream(dat=img)
    bits.h = img.shape[0]
    bits.w = img.shape[1]
    bits.c = img.shape[2]
    entro_coder = codec.entropy_codec(Mode='Arith')
    entro_coder.encode(bits, oname)
    
    size1= os.path.getsize(oname + 'dat')
    size2= os.path.getsize(oname + 'arith')

    #olen, size1, size2 in unit of Byte (assuming input is 8bit int)
    return olen, size1, size2

In [6]:
'''
pref = './Paper_RB_14bit_COMPINTs/CI884/COMPINT'
nx=256
ny=480
ncomp=4
'''


pref = './Paper_RB_14bit_COMPINTs/CI16164/COMPINT'
nx=128
ny=240
ncomp=4


'''
pref = './Paper_RB_14bit_COMPINTs/CI32324/COMPINT'
nx=64
ny=120
ncomp=4
'''

"\npref = './Paper_RB_14bit_COMPINTs/CI32324/COMPINT'\nnx=64\nny=120\nncomp=4\n"

In [7]:
i_f = 3
t_C = read_COMPINT(pref, oc_fname(i_f))

show_data(t_C, 't_C')
for i in range(4):
    show_data(t_C[...,i], 't_C_'+str(i))

t_C shape: (1, 128, 240, 4) , mean = 142.727685546875 , std = 7.031255679262482 , Max = 209.0 , Min = 123.0 , type=float64
t_C_0 shape: (1, 128, 240) , mean = 139.66848958333333 , std = 5.588061288912416 , Max = 184.0 , Min = 131.0 , type=float64
t_C_1 shape: (1, 128, 240) , mean = 145.77099609375 , std = 6.988187479037196 , Max = 209.0 , Min = 128.0 , type=float64
t_C_2 shape: (1, 128, 240) , mean = 145.44319661458334 , std = 8.104334452444068 , Max = 195.0 , Min = 134.0 , type=float64
t_C_3 shape: (1, 128, 240) , mean = 140.02805989583334 , std = 4.327862929677782 , Max = 193.0 , Min = 123.0 , type=float64


In [8]:
idc = 3
data_t = t_C[0,:,:,idc]
data_t = data_t - 128.0
data_t = data_t.astype(np.int8)
show_data(data_t, 'data_t')

data_t shape: (128, 240) , mean = 12.028059895833334 , std = 4.327862929677782 , Max = 65 , Min = -5 , type=int8


In [9]:
oname = 'fAc'
olen, size1, size2 = ar_code(data_t, oname)

print('olen' + str(olen))
print('size1' + str(size1))
print('size2' + str(size2))

ratio1 = float(size1)/float(olen)
ratio2 = float(size2)/float(olen)

print('ratio1= ' + str(ratio1))
print('ratio2= ' + str(ratio2))

olen30720
size130732
size214524
ratio1= 1.000390625
ratio2= 0.472786458333


In [10]:
fs = 1; fe = 4;
nf = fe-fs
size1s = np.zeros([nf,4])
size2s = np.zeros([nf,4])
olens = np.zeros([nf,4])

for i in range(fs,fe):
    print(i)
    t_C = read_COMPINT(pref, oc_fname(i))
    show_data(t_C, '  t_C')
    
    for idc in range(4):
        data_t = t_C[0,:,:,idc]
        data_t = data_t - 128.0
        data_t = data_t.astype(np.int8)
        show_data(data_t, ' channel'+str(idc))
        olen, size1, size2 = ar_code(data_t, oname)

        print('   olen: ' + str(olen) + ' Bytes')
        print('   size1: ' + str(size1) + ' Bytes')
        print('   size2: ' + str(size2) + ' Bytes')

        ratio1 = float(size1)/float(olen)
        ratio2 = float(size2)/float(olen)

        print('ratio1= ' + str(ratio1))
        print('ratio2= ' + str(ratio2))
        
        idx = i-fs
        size1s[idx,idc] = size1
        size2s[idx,idc] = size2
        olens[idx,idc] = olen
    
    print('\n')

1
  t_C shape: (1, 128, 240, 4) , mean = 149.56744791666668 , std = 11.135809717762063 , Max = 210.0 , Min = 117.0 , type=float64
 channel0 shape: (128, 240) , mean = 24.07548828125 , std = 12.423484490430905 , Max = 72 , Min = 0 , type=int8
   olen: 30720 Bytes
   size1: 30732 Bytes
   size2: 19454 Bytes
ratio1= 1.000390625
ratio2= 0.633268229167
 channel1 shape: (128, 240) , mean = 22.438802083333332 , std = 10.283166848787179 , Max = 82 , Min = -11 , type=int8
   olen: 30720 Bytes
   size1: 30732 Bytes
   size2: 18178 Bytes
ratio1= 1.000390625
ratio2= 0.591731770833
 channel2 shape: (128, 240) , mean = 21.484765625 , std = 12.947554052200427 , Max = 67 , Min = -7 , type=int8
   olen: 30720 Bytes
   size1: 30732 Bytes
   size2: 19256 Bytes
ratio1= 1.000390625
ratio2= 0.626822916667
 channel3 shape: (128, 240) , mean = 18.270735677083334 , std = 7.097514812670059 , Max = 61 , Min = -7 , type=int8
   olen: 30720 Bytes
   size1: 30732 Bytes
   size2: 16819 Bytes
ratio1= 1.000390625
rati

In [11]:
for idx in range(nf):
    print(idx+1)
    
    ratio_1 = np.mean(size1s[idx,:])/np.mean(olens[idx,:])
    ratio_2 = np.mean(size2s[idx,:])/np.mean(olens[idx,:])

    #print('   size_orig= ' + str(size_orig))
    print('   ratio_1= ' + str(ratio_1))
    print('   ratio_2= ' + str(ratio_2))
    print(' ')

1
   ratio_1= 1.000390625
   ratio_2= 0.5998291015625
 
2
   ratio_1= 1.000390625
   ratio_2= 0.5824625651041667
 
3
   ratio_1= 1.000390625
   ratio_2= 0.5283203125
 
